In [34]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models
import random

In [35]:
num_classes = 5
learning_rate = 0.01
num_epochs = 10
in_channels = 3

In [37]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(240),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data_path = '/Users/jesseseid/Jseid628/dataset/Hass Avocado Ripening Photographic Dataset/Avocado Ripeness Split /train'
test_data_path = '/Users/jesseseid/Jseid628/dataset/Hass Avocado Ripening Photographic Dataset/Avocado Ripeness Split /test'

# Create an ImageFolder dataset
# This will automatically find the classes (1, 2, 3, 4, 5) from subfolders
train_dataset = datasets.ImageFolder(root=train_data_path, transform=train_transforms)
test_dataset = datasets.ImageFolder(root=test_data_path, transform=test_transforms)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

In [38]:
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

num_ftrs = model.classifier[1].in_features
model.classifier = nn.Linear(num_ftrs, 5)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Model loaded and moved to {device}. Classifier head modified.")
print(model.classifier) # Check the modified classifier

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # Adjust learning rate if needed

# Optional: Learning rate scheduler to reduce LR during training
# Helps in fine-tuning and preventing oscillations
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Reduce LR by 0.1 every 7 epochs

Model loaded and moved to cpu. Classifier head modified.
Linear(in_features=1280, out_features=5, bias=True)


In [39]:
#Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

#Training
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        data, targets = data.to(device), targets.to(device)
        outputs = model(data)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if ((epoch % 1)== 0):
        print(f"Epoch {epoch} Loss: {loss.item()}")

  0%|          | 0/184 [00:01<?, ?it/s]


KeyboardInterrupt: 